In [32]:
import os
import pandas as pd
from textblob import *
import nltk
import numpy as np
from sklearn import tree
import openpyxl
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import *
'''
    ISOT
'''


read_file = r"../data/Des_fake_news/LIAR.xlsx"
read_sheet = "valid"
write_file = r"../data/Des_fake_news/LIAR_PROCESSED_valid.csv"

data = pd.read_excel(read_file, read_sheet)

data = data.dropna(axis=0)
data

,id,class,text,title,speaker,job title,state,party,barelytrues,falses,halftrues,mostlystrues,pantsonfires,context,flag
0,12134.json,barely-true,We have less Americans working now than in the...,"economy,jobs",vicky-hartzler,U.S. Representative,Missouri,republican,1,0,1,0,0,an interview with ABC17 News,fake
2,7891.json,False,Says Having organizations parading as being so...,"campaign-finance,congress,taxes",earl-blumenauer,U.S. representative,Oregon,democrat,0,1,1,1,0,a U.S. Ways and Means hearing,fake
3,8169.json,half-true,Says nearly half of Oregons children are poor.,poverty,jim-francesconi,Member of the State Board of Higher Education,Oregon,none,0,1,1,1,0,an opinion article,fake
4,929.json,half-true,On attacks by Republicans that various program...,"economy,stimulus",barack-obama,President,Illinois,democrat,70,71,160,163,9,interview with CBS News,fake
5,9416.json,False,Says when armed civilians stop mass shootings ...,guns,jim-rubens,Small business owner,New Hampshire,republican,1,1,0,1,0,"in an interview at gun shop in Hudson, N.H.",fake
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1276,7013.json,barely-true,Says U.S. Rep. Charles Bass wants to privatize...,social-security,ann-mclane-kuster,Attorney,New Hampshire,democrat,2,1,3,0,0,"an ad, â€œJanice,â€ released September 6, 2012",fake
1278,2661.json,pants-fire,"In the past two years, Democrats have spent mo...","federal-budget,history",eric-cantor,House Majority Leader,Virginia,republican,9,6,4,4,4,an interview on Comedy Central's Daily Show wi...,fake
1279,3419.json,half-true,"For the first time in more than a decade, impo...","energy,oil-spill,trade",barack-obama,President,Illinois,democrat,70,71,160,163,9,a press conference,fake
1280,12548.json,mostly-true,Says Donald Trump has bankrupted his companies...,candidates-biography,hillary-clinton,Presidential candidate,New York,democrat,40,29,69,76,7,a speech on the economy,real


In [33]:
#remove nonASCIICharacters

data["title"] = data.apply(lambda x: "".join([i for i in x["title"] if ord(i) <= 127]), axis=1)



data["text"] = data.apply(lambda x: "".join([i for i in x["text"] if ord(i) <= 127]), axis=1)


In [45]:
def to_class(s):
    if s == "real":
        return 1
    return 0

In [35]:
data["flag"] = data.apply(lambda x: to_class(x["flag"]), axis=1)
data

,id,class,text,title,speaker,job title,state,party,barelytrues,falses,halftrues,mostlystrues,pantsonfires,context,flag
0,12134.json,barely-true,We have less Americans working now than in the...,"economy,jobs",vicky-hartzler,U.S. Representative,Missouri,republican,1,0,1,0,0,an interview with ABC17 News,0
2,7891.json,False,Says Having organizations parading as being so...,"campaign-finance,congress,taxes",earl-blumenauer,U.S. representative,Oregon,democrat,0,1,1,1,0,a U.S. Ways and Means hearing,0
3,8169.json,half-true,Says nearly half of Oregons children are poor.,poverty,jim-francesconi,Member of the State Board of Higher Education,Oregon,none,0,1,1,1,0,an opinion article,0
4,929.json,half-true,On attacks by Republicans that various program...,"economy,stimulus",barack-obama,President,Illinois,democrat,70,71,160,163,9,interview with CBS News,0
5,9416.json,False,Says when armed civilians stop mass shootings ...,guns,jim-rubens,Small business owner,New Hampshire,republican,1,1,0,1,0,"in an interview at gun shop in Hudson, N.H.",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1276,7013.json,barely-true,Says U.S. Rep. Charles Bass wants to privatize...,social-security,ann-mclane-kuster,Attorney,New Hampshire,democrat,2,1,3,0,0,"an ad, â€œJanice,â€ released September 6, 2012",0
1278,2661.json,pants-fire,"In the past two years, Democrats have spent mo...","federal-budget,history",eric-cantor,House Majority Leader,Virginia,republican,9,6,4,4,4,an interview on Comedy Central's Daily Show wi...,0
1279,3419.json,half-true,"For the first time in more than a decade, impo...","energy,oil-spill,trade",barack-obama,President,Illinois,democrat,70,71,160,163,9,a press conference,0
1280,12548.json,mostly-true,Says Donald Trump has bankrupted his companies...,candidates-biography,hillary-clinton,Presidential candidate,New York,democrat,40,29,69,76,7,a speech on the economy,1


In [36]:
data["text_tb_pol"] = data.apply(lambda x: TextBlob(x["text"]).polarity, axis=1)
data["text_tb_sub"] = data.apply(lambda x: TextBlob(x["text"]).subjectivity, axis=1)

data["title_tb_pol"] = data.apply(lambda x: TextBlob(x["title"]).polarity, axis=1)
data["title_tb_sub"] = data.apply(lambda x: TextBlob(x["title"]).subjectivity, axis=1)
data

,id,class,text,title,speaker,job title,state,party,barelytrues,falses,halftrues,mostlystrues,pantsonfires,context,flag,text_tb_pol,text_tb_sub,title_tb_pol,title_tb_sub
0,12134.json,barely-true,We have less Americans working now than in the...,"economy,jobs",vicky-hartzler,U.S. Representative,Missouri,republican,1,0,1,0,0,an interview with ABC17 News,0,-0.166667,0.066667,0.0,0.0
2,7891.json,False,Says Having organizations parading as being so...,"campaign-finance,congress,taxes",earl-blumenauer,U.S. representative,Oregon,democrat,0,1,1,1,0,a U.S. Ways and Means hearing,0,0.011111,0.055556,0.0,0.0
3,8169.json,half-true,Says nearly half of Oregons children are poor.,poverty,jim-francesconi,Member of the State Board of Higher Education,Oregon,none,0,1,1,1,0,an opinion article,0,-0.283333,0.383333,0.0,0.0
4,929.json,half-true,On attacks by Republicans that various program...,"economy,stimulus",barack-obama,President,Illinois,democrat,70,71,160,163,9,interview with CBS News,0,0.008333,0.191667,0.0,0.0
5,9416.json,False,Says when armed civilians stop mass shootings ...,guns,jim-rubens,Small business owner,New Hampshire,republican,1,1,0,1,0,"in an interview at gun shop in Hudson, N.H.",0,-0.150000,0.400000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1276,7013.json,barely-true,Says U.S. Rep. Charles Bass wants to privatize...,social-security,ann-mclane-kuster,Attorney,New Hampshire,democrat,2,1,3,0,0,"an ad, â€œJanice,â€ released September 6, 2012",0,0.027778,0.222222,0.0,0.0
1278,2661.json,pants-fire,"In the past two years, Democrats have spent mo...","federal-budget,history",eric-cantor,House Majority Leader,Virginia,republican,9,6,4,4,4,an interview on Comedy Central's Daily Show wi...,0,0.010000,0.203333,0.0,0.0
1279,3419.json,half-true,"For the first time in more than a decade, impo...","energy,oil-spill,trade",barack-obama,President,Illinois,democrat,70,71,160,163,9,a press conference,0,0.104167,0.266667,0.0,0.0
1280,12548.json,mostly-true,Says Donald Trump has bankrupted his companies...,candidates-biography,hillary-clinton,Presidential candidate,New York,democrat,40,29,69,76,7,a speech on the economy,1,0.000000,0.000000,0.0,0.0


In [37]:
analyzer = SentimentIntensityAnalyzer()

data["title_vader_scores"] = data.apply(lambda x: analyzer.polarity_scores(x["title"]), axis=1)
data["title_vader_comp"] = data.apply(lambda x: x["title_vader_scores"]["compound"], axis=1)
data["title_vader_neg"] = data.apply(lambda x: x["title_vader_scores"]["neg"], axis=1)
data["title_vader_neu"] = data.apply(lambda x: x["title_vader_scores"]["neu"], axis=1)
data["title_vader_pos"] = data.apply(lambda x: x["title_vader_scores"]["pos"], axis=1)
data = data.drop(["title_vader_scores"], axis=1)

data["text_vader_scores"] = data.apply(lambda x: analyzer.polarity_scores(x["text"]), axis=1)
data["text_vader_comp"] = data.apply(lambda x: x["text_vader_scores"]["compound"], axis=1)
data["text_vader_neg"] = data.apply(lambda x: x["text_vader_scores"]["neg"], axis=1)
data["text_vader_neu"] = data.apply(lambda x: x["text_vader_scores"]["neu"], axis=1)
data["text_vader_pos"] = data.apply(lambda x: x["text_vader_scores"]["pos"], axis=1)
data = data.drop(["text_vader_scores"], axis=1)

In [38]:
def discrete_vader(neg, neu, pos):
    return np.argmax([neg, neu, pos])
    
def discrete_textblob(pol):
    if pol <= -0.33:
        return 0
    elif pol > -0.33 and pol < 0.33:
        return 1
    else:
        return 2

def discrete_textblob_sub(pol):
    if pol < 0.5:
        return 0
    return 1

In [39]:

data["title_vader_class"] = data.apply(lambda x: discrete_vader(x["title_vader_neg"], x["title_vader_neu"], x["title_vader_pos"]), axis=1)
data["text_vader_class"] = data.apply(lambda x: discrete_vader(x["text_vader_neg"], x["text_vader_neu"], x["text_vader_pos"]), axis=1)

data["text_tb_pol_class"] = data.apply(lambda x: discrete_textblob(x["text_tb_pol"]), axis=1)
data["text_tb_sub_class"] = data.apply(lambda x: discrete_textblob_sub(x["text_tb_sub"]), axis=1)

data["title_tb_pol_class"] = data.apply(lambda x: discrete_textblob(x["title_tb_pol"]), axis=1)
data["title_tb_sub_class"] = data.apply(lambda x: discrete_textblob_sub(x["title_tb_sub"]), axis=1)
data


,id,class,text,title,speaker,job title,state,party,barelytrues,falses,...,text_vader_comp,text_vader_neg,text_vader_neu,text_vader_pos,title_vader_class,text_vader_class,text_tb_pol_class,text_tb_sub_class,title_tb_pol_class,title_tb_sub_class
0,12134.json,barely-true,We have less Americans working now than in the...,"economy,jobs",vicky-hartzler,U.S. Representative,Missouri,republican,1,0,...,0.0000,0.000,1.000,0.000,1,1,1,0,1,0
2,7891.json,False,Says Having organizations parading as being so...,"campaign-finance,congress,taxes",earl-blumenauer,U.S. representative,Oregon,democrat,0,1,...,-0.3400,0.072,0.928,0.000,1,1,1,0,1,0
3,8169.json,half-true,Says nearly half of Oregons children are poor.,poverty,jim-francesconi,Member of the State Board of Higher Education,Oregon,none,0,1,...,-0.4767,0.307,0.693,0.000,0,1,1,0,1,0
4,929.json,half-true,On attacks by Republicans that various program...,"economy,stimulus",barack-obama,President,Illinois,democrat,70,71,...,-0.4404,0.083,0.917,0.000,1,1,1,0,1,0
5,9416.json,False,Says when armed civilians stop mass shootings ...,guns,jim-rubens,Small business owner,New Hampshire,republican,1,1,...,-0.8750,0.345,0.655,0.000,1,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1276,7013.json,barely-true,Says U.S. Rep. Charles Bass wants to privatize...,social-security,ann-mclane-kuster,Attorney,New Hampshire,democrat,2,1,...,0.3400,0.000,0.789,0.211,1,1,1,0,1,0
1278,2661.json,pants-fire,"In the past two years, Democrats have spent mo...","federal-budget,history",eric-cantor,House Majority Leader,Virginia,republican,9,6,...,0.0000,0.000,1.000,0.000,1,1,1,0,1,0
1279,3419.json,half-true,"For the first time in more than a decade, impo...","energy,oil-spill,trade",barack-obama,President,Illinois,democrat,70,71,...,0.0000,0.000,1.000,0.000,1,1,1,0,1,0
1280,12548.json,mostly-true,Says Donald Trump has bankrupted his companies...,candidates-biography,hillary-clinton,Presidential candidate,New York,democrat,40,29,...,0.0000,0.000,1.000,0.000,1,1,1,0,1,0


In [40]:
#data.to_csv(r"../data/Des_fake_news/ISOT_PROCESSED.csv", index=False)

In [41]:
'''
    Begin Neural Net for news text
'''

#data = pd.read_csv(r"../data/Des_fake_news/ISOT_PROCESSED.csv")
'''text only'''
features = data[["text_tb_pol", "text_vader_pos", "text_vader_neg", "text_vader_neu"]].to_numpy()


import tensorflow as tf
from sklearn.model_selection import train_test_split

imdb_model = tf.keras.models.load_model(r"SA_imdb/IMDB_NN.keras")
imdb_sentiment = np.argmax(imdb_model.predict(features), axis=-1)

tweets_model = tf.keras.models.load_model(r"SA_tweets/TWEETS_NN.keras")
tweets_sentiment = np.argmax(tweets_model.predict(features), axis=-1)

data["text_NN_imdb"] = imdb_sentiment
data["text_NN_tweets"] = tweets_sentiment

#data.to_csv(r"../data/Des_fake_news/ISOT_PROCESSED.csv", index=False)

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [42]:
'''
    Begin Neural Net for news title
'''


'''text only'''
features = data[["title_tb_pol", "title_vader_pos", "title_vader_neg", "title_vader_neu"]].to_numpy()


import tensorflow as tf
from sklearn.model_selection import train_test_split

imdb_model_two = tf.keras.models.load_model(r"SA_imdb/IMDB_NN.keras")
imdb_sentiment_two = np.argmax(imdb_model_two.predict(features), axis=-1)

tweets_model_two = tf.keras.models.load_model(r"SA_tweets/TWEETS_NN.keras")
tweets_sentiment_two = np.argmax(tweets_model_two.predict(features), axis=-1)

data["title_NN_imdb"] = imdb_sentiment_two
data["title_NN_tweets"] = tweets_sentiment_two

#data.to_csv(r"../data/Des_fake_news/ISOT_PROCESSED.csv", index=False)

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [43]:
'''
    Begin log regression for text
'''
from sklearn.linear_model import LogisticRegression
import pickle

texts = data[["text_tb_pol", "text_vader_pos", "text_vader_neg", "text_vader_neu"]].to_numpy()

tweet_text_log:  LogisticRegression = None
with open(r"SA_tweets/TWEETS_LOG.pkl", "rb") as file:
    tweet_text_log = pickle.load(file)

imdb_text_log:  LogisticRegression = None
with open(r"SA_imdb/IMDB_LOG.pkl", "rb") as file:
    imdb_text_log = pickle.load(file)

data["text_log_imdb"] = imdb_text_log.predict(texts)
data["text_log_tweets"] = imdb_text_log.predict(texts)





In [44]:
'''
    Begin log regression for title
'''
from sklearn.linear_model import LogisticRegression
import pickle

titles = data[["title_tb_pol", "title_vader_pos", "title_vader_neg", "title_vader_neu"]].to_numpy()


tweet_text_log:  LogisticRegression = None
with open(r"SA_tweets/TWEETS_LOG.pkl", "rb") as file:
    tweet_text_log = pickle.load(file)

imdb_text_log:  LogisticRegression = None
with open(r"SA_imdb/IMDB_LOG.pkl", "rb") as file:
    imdb_text_log = pickle.load(file)

data["title_log_imdb"] = imdb_text_log.predict(texts)
data["title_log_tweets"] = imdb_text_log.predict(texts)


data.to_csv(write_file, index=False)